# Chapter 2. Stateless Web Application

## What is Stateless?

REST는 복수의 아키텍쳐 스타일을 조합한 것이며, 그 중에 Stateless Server가 있다.

![](http://localhost:8888/files/rest.png)

![](http://localhost:8888/files/rest_style.png)

## Stateless Server

![](http://localhost:8888/files/stateless.png)

## Cache

![](http://localhost:8888/files/cache.png)

- 그림출처1: http://reimaginer.tistory.com/35
- 그림출처2: http://blog.naver.com/lsh00124/220393694599

![](http://localhost:8888/files/stateless_comm.png)

![](http://localhost:8888/files/pros_and_cons.png)

출처: http://blog.naver.com/lsh00124/220404390444

# Placeholder Image Server

- Placeholder image service는 일반적으로 이미지 사이즈가 있는 URL을 취하고 이를 통해 이미지를 생성한다.

## 1. 프로젝트 디렉토리 생성

`$ django-admin.py startproject placeholder --template=project_name`

## 2. View 생성

1. 요청한 사이즈를 기반으로 placeholder image를 렌더링하는 뷰
2. Home 화면에 보여줄 내용을 렌더링하는 뷰(기존에 만들어져있음)

In [1]:
def placeholder(request, width, height):
    return HttpResponse('Ok')

## 3. URL Pattern

- placeholder 함수는 width, height 두 개의 인자를 가지고 있으며
- 이는 URL을 통해 전달된다.

In [ ]:
urlpatterns = (
    url(r'^image/(?P<width>[0-9]+)x(?P<height>[0-9]+)/$', placeholder,
        name="placeholder"),
    url(r'', index, name="homepage"),
)


## 4. URL로 넘어온 값에 대한 Validation with Django forms

- URL에서 string으로 넘어왔던 값이 form의 validation을 통해서 integer로 converting

In [ ]:
from django import forms

class ImageForm(forms.Form):
    """Form to validate requested placeholder image."""
    
    height = forms.IntegerField(min_value=1, max_value=2000)
    width = forms.IntegerField(min_value=1, max_value=2000)

def placeholder(request, width, height):
    form = ImageForm({'width': width, 'height': height})
    if form.is_valid():
        width = form.cleaned_data('width')
        height = form.cleaned_data('height')
    return HttpResponse('Ok')


## 5. validation이 실패했을 경우 400 Bad Request 응답해주기

In [ ]:
from django.http import HttpResponse, HttpResponseBadRequest
from django import forms

class ImageForm(forms.Form):
    """Form to validate requested placeholder image."""
    
    height = forms.IntegerField(min_value=1, max_value=2000)
    width = forms.IntegerField(min_value=1, max_value=2000)

def placeholder(request, width, height):
    form = ImageForm({'width': width, 'height': height})
    if form.is_valid():
        width = form.cleaned_data('width')
        height = form.cleaned_data('height')
        return HttpResponse('Ok')
    else:
        return HttpResponseBadRequest('Invalid Image Request')

- 아직 이미지를 생성시키는 것은 구현시키지 않았음


## 6. 이미지 생성을 위한 패키지 설치 - Pillow

`$ pip install Pillow`


## 7. 이미지 생성을 위해 ImageForm에 메소드 추가 (to encapsulate the logic)

- Pillow: 튜플 형태의 2개 인자값 + 색상
    1. color mode ex) RGB
    2. 이미지 사이즈
    3. 색상 (default: black)

In [ ]:
from io import BytesIO
from PIL import Image

class ImageForm(forms.Form):
    ...
    def generate(self, image_formate='PNG'):
        height = self.cleaned_data['height']
        width = self.cleaned_data['width']
        image = Image.new('RGB', (width, height))
        
        content = BytesIO()
        image.save(content, image_format)
        content.seek(0)
        return content

def placeholder(request, width, height):
    form = ImageForm({'width': width, 'height': height})
    if form.is_valid():
        image = form.generate()
        return HttpReponse(image, content_type='image/png')
    else:
        return HttpResponseBadRequest('Invalid Image Request')

- io.BytesIO.seek()
    - https://docs.python.org/3/library/io.html#io.IOBase.seek
    - Change the stream position
    - SEEK_SET or 0 – start of the stream (the default); offset should be zero or positive
    - SEEK_CUR or 1 – current stream position; offset may be negative
    - SEEK_END or 2 – end of the stream; offset is usually negative

## 8. 이미지에 텍스트 새기기

In [ ]:
from PIL import Image, ImageDraw

def generate(self, image_format='PNG'):
    ...
    image = Image.new('RGB', (width, height))
    
    draw = ImageDraw.Draw(image)
    text = '{} x {}'.format(width, height)
    textwidth, textheight = draw.textsize(text)
    if textwidth < width and textheight < height:
        texttop = (height - textheight) // 2
        textleft = (width - textwidth) // 2
        draw.text((textleft, texttop), text, fill=(255, 255, 255))
    
    content = BytesIO()
    ...

## 9. 캐시

- 이 서비스는 불필요한 request를 서버에 계속해서 보내고 있다.
- 이를 피하기 위해 캐시를 사용하며 두 가지 옵션이 있다.
    1. Server-side
        - Django's cache utilities
            - `from django.core.cache import cache`
                - `cache.get(key)`
                - `cache.set(key, content, 시간)`
        - Memcached
        - Redis
        - etc.
    2. Client-side
        - browser's built-in caching
            - etag
                1. middleware
                    - USE_ETAGS setting
                    - md5 hash
                    - etag header 정보를 생성하는데 필요한 모든 작업을 위해 view 함수가 필요
                2. etag decorator
                    - single argument, which is a function to generate the ETag header
                    - sha hash
                    - middleware와는 다르게 view 함수가 calling되기 전에 etag를 생성한다는 이점이 있다.
                    - 이는 processing time과 resource를 절약해준다.

### etag

- ETag는 HTTP 1.1 호환 웹 서버가 반환하는 응답 헤더로 리소스가 변경 됐는지 확인하는데 필요한 정보를 제공합니다.
- 캐싱 기능이 있는 애플리케이션(ex 브라우저)에서 유용
- 이 태그를 참조하여 이전에 받았던 ETag와 동일하다면 불필요하게 리소스를 다운로드 하는 작업을 줄일 수 있습니다.
- ETag의 값은 파일 크기와 파일을 수정한 날짜를 기반으로 만듬

**Overview**

![](http://localhost:8888/files/http_caching_1.png)

**When an item is fully cached**

![](http://localhost:8888/files/fully_cached.png)

- For instance, once **CSS stylesheets** from your application are downloaded by the browser there’s no need to download them again during the user’s session.
- This holds true for many asset types like javascript files, images and even infrequently changing dynamic content.
- In these instances it is beneficial for the users browser to cache this file locally, and use that copy whenever the resource is requested again.
- An application using HTTP cache headers is able to control this caching behavior and **alleviate server-side load**.

**Conditional Requests**

![](http://localhost:8888/files/conditional_requests.png)

Conditional requests are those where the browser can ask the server if it has an updated copy of the resource. The browser will send some information about the cached resource it holds and the server will determine whether updated content should be returned or the browser’s copy is the most recent. In the case of the latter an HTTP status of 304 (not modified) is returned.

출처: https://devcenter.heroku.com/articles/increasing-application-performance-with-http-cache-headers

**다른 그림**

![](http://localhost:8888/files/http-cache-control.png)

**ETag Response Header에는 어떤 정보가 있을까**

![](http://localhost:8888/files/covert-timing-channels-using-http-cache-headers.jpg)

**책의 예제는 Servier-side and Client-side Caching 모두 사용하고 있다.**

---

# 이제부터 Home 화면을 꾸며보자!

1. template, static setting
2. html, css 만들기
3. view 만들기

## 1. template, static setting

## 2. html, css 만들기

## 3. view 만들기